In [1]:
import requests
from bs4 import BeautifulSoup as bs
from openpyxl import Workbook
from openpyxl.drawing.image import Image
from openpyxl.utils.exceptions import IllegalCharacterError
import urllib.request
import urllib.error
import time

In [2]:
class TapTapScraper:
    def __init__(self):
        self.wb = Workbook()
        self.ws = self.wb.active
        self.ws.title = "TapTap"
        self.next_row = 1
        self.row_dict = {1: 1}

    def parse_page(self, page, page_num):
        if page_num in self.row_dict.keys():
            self.next_row = self.row_dict[page_num]
        while page.status_code != 200:
            time.sleep(1)
            page = get_page(page_num)
        page_soup = bs(page.text, 'html.parser')
        commentList = page_soup.find_all(class_="posts-item-text topic-posts-item-text")
        for comment in commentList:
            parsed = self.parse_comment(comment)
            if len(parsed[3]) < 40:
                continue
            text_data, img_data = list(parsed[:-1]), parsed[-1]
            for i in range(4):
                try:
                    self.ws.cell(row=self.next_row, column=(i + 1), value=text_data[i])
                except IllegalCharacterError:
                    pass
            col = 6
            self.ws.cell(row=self.next_row, column=5, value=len(img_data))
            while img_data:
                # save image
                img_url = img_data.pop(0)
                ext = "gif" if img_url.find('.gif') != -1 else "jpg"
                urllib.request.urlretrieve(img_url,
                                           "/Users/yuhongc/PycharmProjects/SK_Weapon_Event/TapTap_Pictures/%s-%s.%s" % (
                                           parsed[2], col - 5, ext))
                # insert to worksheet
                self.ws.cell(row=self.next_row, column=col, value=img_url)
                col += 1
            self.next_row += 1
        self.row_dict[page_num + 1] = self.next_row
        time.sleep(1)

    def parse_comment(self, comment):
        comment_soup = bs(str(comment), 'html.parser')

        """
        userList = comment_soup.find_all(class_="taptap-user")  # list of users associated with this comment
        str(userList[0]).split('"')[3]  # first user is original commenter 
        """
        tap_user_id = str(comment_soup.find_all(class_='taptap-user')[0]).split('"')[3]
        tap_user_name = comment_soup.find_all(class_="taptap-user-name taptap-link")[0].get_text()
        floor_num = int(comment_soup.find_all(class_="pull-right")[0].get_text()[:-1])
        comment_body = comment_soup.find_all(class_="item-text-body bbcode-body js-open-bbcode-image")[0]
        comment_text = comment_body.prettify()
        comment_text = comment_text.replace("<br/>", "")
        comment_text = remove_tags(comment_text)
        comment_img_srcs = [str(img)[str(img).find("src"):].split('"')[1]
                            for img in comment_body.find_all("img")]  # convert to str then locate src

        return tap_user_id, tap_user_name, floor_num, comment_text, comment_img_srcs

    def save(self, checkpoint=None):
        if not checkpoint:
            self.wb.save('TapTap.xlsx')
        else:
            self.wb.save('TapTap-checkpoint-%d.xlsx' % checkpoint)

def get_page(page_num):
    url = "https://www.taptap.com/topic/3341168?page=%d#postsList" % page_num
    page = requests.get(url)
    return page

def remove_tags(text):
    while "<" in text:
        start = text.find("<")
        end = text.find(">")
        text = text[:start] + text[end + 1:]
    return text


In [21]:
page = get_page(146)
page_soup = bs(page.text, 'html.parser')

In [14]:
commentList = page_soup.find_all(class_="posts-item-text topic-posts-item-text")
# print(commentList[0])
comment = commentList[0]
comment_soup = bs(str(comment), 'html.parser')
#print(comment_soup.prettify())
parsed = Scraper.parse_comment(comment)
parsed[-1][0]

'https://img2.tapimg.com/bbcode/images/8c60997d158241108fb3756399a8ea27.png?imageView2/2/w/1320/q/80/format/jpg/interlace/1/ignore-error/1'

In [3]:
Scraper = TapTapScraper()
i=1

In [4]:
while i < 160:
    try:
        Scraper.parse_page(get_page(i), i)
        print("Scraped page %d" % i)
        Scraper.save()
        i += 1
    except urllib.error.HTTPError as e:
        print("HTTPError detected, re-scraping page")
        continue
    except ConnectionError as e:
        time.sleep(20)
        continue
    if i%30 == 0:
        Scraper.save(i)


Scraped page 1


Scraped page 2


Scraped page 3


Scraped page 4


Scraped page 5


...

Scraped page 155


Scraped page 156


Scraped page 157


Scraped page 158


Scraped page 159


In [5]:
urllib.request.urlretrieve("https://img2.tapimg.com/bbcode/images/7fd41fd8afea4dc7939ce5a5a1673b66.gif", "local-filename.jpg")
img = Image('local-filename.jpg')
# Scraper.ws.add_image(img, 'D5')


